# 决策集解析（XML2DataFrame）
rule_xml2df(xml_path)：决策集遍历规则转DataFrame，包括AND、OR等逻辑嵌套，并通过Depth记录逻辑层次
* RuleName：规则编码
* Enabled：是否启用
* ConditionType：条件类型
* VariableCategory：变量类别
* Variable：变量名
* VariableLabel：变量标题
* Datatype：数据类型
* Value：值
* LogicalOperator：逻辑运算符（AND、OR）
* Depth：逻辑层次

In [1]:
import xml.etree.ElementTree as ET
import pandas as pd

def parse_conditions(element, parent_op=None, depth=0):
    """递归解析条件，包括<and>和<or>操作"""
    conditions = []

    # 如果是<and>或<or>，我们记录它，并继续解析它的子元素
    if element.tag in ['and', 'or']:
        current_op = element.tag
        for child in element:
            conditions.extend(parse_conditions(child, current_op, depth + 1))
    
    # 如果是<atom>，我们提取<atom>的信息
    elif element.tag == 'atom':
        condition_type = element.get('op')
        left_var_category = element.find('left').get('var-category')
        left_var = element.find('left').get('var')
        left_var_label = element.find('left').get('var-label')
        left_datatype = element.find('left').get('datatype')
        
        # 判断是const还是content，并检查value_element是否存在
        value_element = element.find('value')
        if value_element is not None:
            if 'const' in value_element.attrib:
                value = value_element.get('const')
            else:
                value = value_element.get('content')
        else:
            value = None
        
        # 将条件信息添加到列表
        conditions.append({
            'ConditionType': condition_type,
            'VariableCategory': left_var_category,
            'Variable': left_var,
            'VariableLabel': left_var_label,
            'Datatype': left_datatype,
            'Value': value,
            'LogicalOperator': parent_op,  # 记录<and>或<or>操作符
            'Depth': depth  # 记录层次深度，用于表示嵌套
        })
    
    return conditions


def rule_xml2df(xml_path):
    # 解析XML
    root = ET.parse(xml_path).getroot()
    
    # 初始化DataFrame的列
    columns = ['RuleName', 'Enabled', 'ConditionType', 'VariableCategory', 
               'Variable', 'VariableLabel', 'Datatype', 'Value', 'LogicalOperator', 'Depth']
    rules_data = []

    # 提取rule信息
    for rule in root.findall('rule'):
        rule_name = rule.get('name')
        rule_enabled = rule.get('enabled')
        
        # 提取条件块（if block中的<and>和<or>）
        for condition in rule.findall('.//if/*'):  # 查找<and>或<or>
            conditions = parse_conditions(condition)
            for cond in conditions:
                cond.update({
                    'RuleName': rule_name,
                    'Enabled': rule_enabled,
                })
                rules_data.append(cond)

        # 提取then block中的var-assign信息
        for var_assign in rule.findall('.//var-assign'):
            var_category = var_assign.get('var-category')
            var = var_assign.get('var')
            var_label = var_assign.get('var-label')
            datatype = var_assign.get('datatype')
            value_element = var_assign.find('value')
            
            # 判断是const还是content，并检查value_element是否存在
            if value_element is not None:
                if 'const' in value_element.attrib:
                    value = value_element.get('const')
                else:
                    value = value_element.get('content')
            else:
                value = None
            
            rules_data.append({
                'RuleName': rule_name,
                'Enabled': rule_enabled,
                'ConditionType': 'Action',
                'Variable Category': var_category,
                'Variable': var,
                'VariableLabel': var_label,
                'Datatype': datatype,
                'Value': value,
                'LogicalOperator': None,
                'Depth': 0
            })

    # 创建DataFrame
    df = pd.DataFrame(rules_data, columns=columns)
    return df


# 规则自动打标&通过率测算

1、先用Excel2XML生成新版本规则的XML，包括下线、新增和调整阈值等都在XML里修改。   
2、然后再用rule_xml2df读入新版策略的XML解析成DataFrame再生成Desc再进行打标。    
——这样可以避免新规则手动输入的Desc不符合规范，且测算完成后可以实现即刻部署。 

生成规则描述函数：  
generate_rule_descriptions(rule)：
* rule：rule_info（通过xml解析生成，并根据需求去除结果输出、对照条件等不需要包含在描述中的条件）

自动打标函数：   
result_df, success_df, failed_rules= apply_rules(base_df, rule_df)
* base_df：数据底表
* rule_df：需要自动打标的规则信息（用generate_rule_descriptions()生成）

## rule_info生成Desc文本

In [2]:
import re

# 条件符号字典
var_options = {
    'GreaterThen': '>',
    'GreaterThenEquals': '>=',
    'LessThen': '<',
    'LessThenEquals': '<=',
    'Equals': '==',
    'NotEquals': '!=',
    'In': 'in',
    'NotIn': 'not in'
}

def generate_rule_descriptions(df):
    # 初始化新的列表用于存储规则描述
    rule_descriptions = []

    # 区间检测的正则表达式，匹配区间格式
    interval_pattern = re.compile(r'^(\(\d+(\.\d+)?,\d+(\.\d+)?\)|\[\d+(\.\d+)?,\d+(\.\d+)?\]|\[\d+(\.\d+)?,\d+(\.\d+)?\)|\(\d+(\.\d+)?,\d+(\.\d+)?\])$')
    
    # 正则表达式匹配负数和浮点数
    is_number = re.compile(r'^-?\d+(\.\d+)?$')

    # 遍历每个规则
    for rule_name, group in df.groupby('RuleName'):
        # 按 Depth 和 LogicalOperator 进行排序
        group = group.sort_values(by=['Depth'])

        # 使用字典保存不同层次的条件组
        depth_conditions = {}

        # 遍历当前规则的每一行
        for _, row in group.iterrows():
            condition_type = row['ConditionType']
            depth = row['Depth']
            logical_operator = row['LogicalOperator']

            # 检查 Condition Type 是否在字典中
            if condition_type in var_options:
                operator = var_options[condition_type]

                # 如果 Value 为 None，跳过此条件并将其标记为人工规则
                if row['Value'] is None:
                    depth_conditions[depth] = ["人工规则"]
                    break
                else:
                    # 处理Value值
                    value = row['Value'].strip()

                if condition_type in ['In', 'NotIn']:
                    # 处理 In 或 NotIn 的条件
                    values = value.split(',')  # 将 Value 按逗号分隔
                    quoted_values = []
                    temp_value = ''  # 用于暂存被分割的区间

                    for v in values:
                        v = v.strip()
                        if interval_pattern.match(v):  # 如果是区间，直接加入
                            quoted_values.append(f"'{v}'")
                        else:
                            if not v.startswith('(') and not v.startswith('[') and not v.endswith(']') and not v.endswith(')'):
                                # 如果不是区间中的一部分，直接处理
                                quoted_values.append(f"'{v}'")
                            else:
                                # 如果是区间的开始或结束部分，暂存
                                if temp_value:
                                    temp_value += f",{v}"  # 拼接区间
                                    quoted_values.append(f"'{temp_value}'")  # 将拼接后的区间加入
                                    temp_value = ''  # 清空暂存
                                else:
                                    temp_value = v  # 记录区间的起始部分

                    value = f"({', '.join(quoted_values)})"  # 用括号包裹多个值
                elif is_number.match(value):  # 改为匹配数字，包括负数和浮动数
                    # 如果是数字（包括负数或浮动数），不加引号
                    pass
                else:
                    value = f"'{value}'"  # 其他情况加引号

                # 生成条件表达式
                condition = f"{row['Variable'].lower()} {operator} {value}"
                
                # 将条件添加到相应深度的列表中
                if depth not in depth_conditions:
                    depth_conditions[depth] = []
                depth_conditions[depth].append(condition)
            else:
                # 如果 ConditionType 不在字典中，则跳过处理
                depth_conditions[depth] = ["人工规则"]
                break

        # 构建规则描述
        desc_parts = []
        max_depth = max(depth_conditions.keys(), default=-1)  # 获取最大深度，处理无条件情况

        for depth in range(max_depth, -1, -1):  # 从最大深度开始往上合并
            if depth in depth_conditions:
                # 获取当前深度的条件组
                conditions = depth_conditions[depth]
                
                # 判断当前层次是否有逻辑运算符
                operator = group[group['Depth'] == depth]['LogicalOperator'].iloc[0] if not group[group['Depth'] == depth].empty else 'and'
                
                if depth == 1:
                    # 顶层不需要括号，直接组合
                    if operator == 'or':
                        desc_parts.append(' or '.join(conditions))
                    else:
                        desc_parts.append(' and '.join(conditions))
                else:
                    # 根据运算符组合条件
                    if operator == 'or':
                        desc_parts.append(f"({' or '.join(conditions)})")
                    else:
                        desc_parts.append(f"({' and '.join(conditions)})")

        # 最终描述
        final_desc = ' and '.join(desc_parts) if desc_parts else "人工规则"
        rule_descriptions.append({'RuleName': rule_name, 'Desc': final_desc})

    # 创建新的 DataFrame
    desc_df = pd.DataFrame(rule_descriptions)
    return desc_df


这里需要导入需要打标的规则集，常用场景如：  
1、调整后的规则集（包括新增、调整阈值和下线）  
2、其他渠道的规则在你的样本上打标进行效果检验

### 将需要打标的决策集转为DataFrame

In [3]:
path = r'D:\shidejing\榕树\0.初版策略\单变量分析'.replace('\\','/')+'/'
path

'D:/shidejing/榕树/0.初版策略/单变量分析/'

In [65]:
rule_info = rule_xml2df(path+'榕树现有三五人行.xml')
rule_info['Enabled'] = rule_info['Enabled'].fillna('true')
rule_cal = rule_info[rule_info['ConditionType']!='Action'].copy()
rule_cal

,RuleName,Enabled,ConditionType,VariableCategory,Variable,VariableLabel,Datatype,Value,LogicalOperator,Depth
0,B_F_SX_THIRD_RULE_00001,true,Equals,百融数据采集,br_flag_executionlimited,百融-法院被执行人限高版产品输出标识,Integer,1,and,1
3,B_F_SX_THIRD_RULE_00002,true,In,风控信息,phoneShutDown,停机次数（新）,String,"(-1,-1],(5,+],(5,+)",and,1
6,B_F_SX_THIRD_RULE_00003,true,GreaterThenEquals,百融数据采集,br_als_m1_id_caon_allnum,百融-按身份证号查询，近1个月申请线上现金分期的次数,Integer,16,and,1
7,B_F_SX_THIRD_RULE_00003,true,GreaterThenEquals,百行数据采集,bh_RevremainingAmount_sum,百行-循环-未结清总余额,BigDecimal,3653,and,1
8,B_F_SX_THIRD_RULE_00003,true,In,客户授信信息,channelPromote,渠道-推广渠道,String,"融360,洋钱罐,OPPO,优速贷,花鸭,AI营销,百融,拍拍贷联营,数票通,同程半流程,点...",and,1
...,...,...,...,...,...,...,...,...,...,...
2455,CREDIT_PEOPLE_BANK_RULE_PB_00253_T,true,GreaterThen,风控信息,rsPbankScoreV2411,榕树人行模型V2411,BigDecimal,0.056194,and,1
2456,CREDIT_PEOPLE_BANK_RULE_PB_00253_T,true,GreaterThenEquals,风控信息,AF_score,微言反欺诈评分,Integer,0,and,1
2457,CREDIT_PEOPLE_BANK_RULE_PB_00253_T,true,LessThenEquals,风控信息,AF_score,微言反欺诈评分,Integer,611,and,1
2458,CREDIT_PEOPLE_BANK_RULE_PB_00253_T,true,In,客户授信信息,channelPromote,渠道-推广渠道,String,百融,and,1


In [66]:
#去掉不启用的规则&对照规则
#rule_cal = rule_cal[rule_cal['Enabled']=='true']
rule_cal = rule_cal[rule_cal['RuleName'].str[-1:] != 'T']
rule_cal

,RuleName,Enabled,ConditionType,VariableCategory,Variable,VariableLabel,Datatype,Value,LogicalOperator,Depth
0,B_F_SX_THIRD_RULE_00001,true,Equals,百融数据采集,br_flag_executionlimited,百融-法院被执行人限高版产品输出标识,Integer,1,and,1
3,B_F_SX_THIRD_RULE_00002,true,In,风控信息,phoneShutDown,停机次数（新）,String,"(-1,-1],(5,+],(5,+)",and,1
6,B_F_SX_THIRD_RULE_00003,true,GreaterThenEquals,百融数据采集,br_als_m1_id_caon_allnum,百融-按身份证号查询，近1个月申请线上现金分期的次数,Integer,16,and,1
7,B_F_SX_THIRD_RULE_00003,true,GreaterThenEquals,百行数据采集,bh_RevremainingAmount_sum,百行-循环-未结清总余额,BigDecimal,3653,and,1
8,B_F_SX_THIRD_RULE_00003,true,In,客户授信信息,channelPromote,渠道-推广渠道,String,"融360,洋钱罐,OPPO,优速贷,花鸭,AI营销,百融,拍拍贷联营,数票通,同程半流程,点...",and,1
...,...,...,...,...,...,...,...,...,...,...
2447,CREDIT_PEOPLE_BANK_RULE_PB_00253,true,GreaterThen,风控信息,rsPbankScoreV2411,榕树人行模型V2411,BigDecimal,0.056194,and,1
2448,CREDIT_PEOPLE_BANK_RULE_PB_00253,true,GreaterThenEquals,风控信息,AF_score,微言反欺诈评分,Integer,0,and,1
2449,CREDIT_PEOPLE_BANK_RULE_PB_00253,true,LessThenEquals,风控信息,AF_score,微言反欺诈评分,Integer,611,and,1
2450,CREDIT_PEOPLE_BANK_RULE_PB_00253,true,In,客户授信信息,channelPromote,渠道-推广渠道,String,百融,and,1


In [67]:
#去掉对照规则的条件
contrast_condition=['正常组灰度组区分']
for i in range(50):
    variable_label='人行规则10等分人固定随机数'+str(i+1)
    contrast_condition.append(variable_label)

rule_cal = rule_cal[~rule_cal['VariableLabel'].isin(contrast_condition)].copy()


#某些特殊配置或分流类随机规则去掉随机数后会错误命中，手动除去
#rule_cal = rule_cal[rule_cal['RuleName']!='B_F_SX_FIVE_RULE_Random_00001'].copy()
rule_cal

,RuleName,Enabled,ConditionType,VariableCategory,Variable,VariableLabel,Datatype,Value,LogicalOperator,Depth
0,B_F_SX_THIRD_RULE_00001,true,Equals,百融数据采集,br_flag_executionlimited,百融-法院被执行人限高版产品输出标识,Integer,1,and,1
3,B_F_SX_THIRD_RULE_00002,true,In,风控信息,phoneShutDown,停机次数（新）,String,"(-1,-1],(5,+],(5,+)",and,1
6,B_F_SX_THIRD_RULE_00003,true,GreaterThenEquals,百融数据采集,br_als_m1_id_caon_allnum,百融-按身份证号查询，近1个月申请线上现金分期的次数,Integer,16,and,1
7,B_F_SX_THIRD_RULE_00003,true,GreaterThenEquals,百行数据采集,bh_RevremainingAmount_sum,百行-循环-未结清总余额,BigDecimal,3653,and,1
8,B_F_SX_THIRD_RULE_00003,true,In,客户授信信息,channelPromote,渠道-推广渠道,String,"融360,洋钱罐,OPPO,优速贷,花鸭,AI营销,百融,拍拍贷联营,数票通,同程半流程,点...",and,1
...,...,...,...,...,...,...,...,...,...,...
2434,CREDIT_PEOPLE_BANK_RULE_PB_00252,true,In,客户授信信息,channelPromote,渠道-推广渠道,String,百融,and,1
2447,CREDIT_PEOPLE_BANK_RULE_PB_00253,true,GreaterThen,风控信息,rsPbankScoreV2411,榕树人行模型V2411,BigDecimal,0.056194,and,1
2448,CREDIT_PEOPLE_BANK_RULE_PB_00253,true,GreaterThenEquals,风控信息,AF_score,微言反欺诈评分,Integer,0,and,1
2449,CREDIT_PEOPLE_BANK_RULE_PB_00253,true,LessThenEquals,风控信息,AF_score,微言反欺诈评分,Integer,611,and,1


In [68]:
rule_list=rule_info[rule_info['VariableLabel']=='规则结果描述'][['RuleName','Enabled','Value']].rename(columns={'Value': 'Content'}).reset_index(drop=True)
#去掉不启用的规则&对照规则
#rule_list = rule_list[rule_list['Enabled']=='true']
rule_list = rule_list[rule_list['RuleName'].str[-1:] != 'T']
rule_list

,RuleName,Enabled,Content
0,B_F_SX_THIRD_RULE_00001,true,百融限高版拒绝
1,B_F_SX_THIRD_RULE_00002,true,拒绝-数盒魔方-停机次数
2,B_F_SX_THIRD_RULE_00003,true,"（融360,数票通,洋钱罐,百融,拍拍贷联营,OPPO ,优速贷,花鸭,AI营销,同程半流程..."
4,B_F_SX_THIRD_RULE_00004,true,"（融360,数票通,洋钱罐,百融,拍拍贷联营,OPPO ,优速贷,花鸭,AI营销,同程半流程..."
6,B_F_SX_THIRD_RULE_00005,true,"（融360,数票通,洋钱罐,百融,拍拍贷联营,OPPO,优速贷,花鸭,AI营销,同程半流程,..."
...,...,...,...
333,B_F_SX_SECOND_RULE_00006,true,百行-循环-历史最严重逾期状态（过去1080 天）为M2/M3/M4/M5/M6/M6/M7
334,B_F_SX_SECOND_RULE_00007,true,百行-非循环-历史最严重逾期状态（过去1080 天）为M2/M3/M4/M5/M6/M7
335,B_F_SX_SECOND_RULE_00008,true,百行-金融租赁-历史最严重逾期状态（过去1080 天）为M2/M3/M4/M5/M6/M7
336,CREDIT_PEOPLE_BANK_RULE_PB_00252,true,(百融)海纳-风险等级指数 > 3 & 榕树申请评分模型V2411 > 0.0599 & 码...


### 自动生成规则Desc

In [69]:
desc_df = generate_rule_descriptions(rule_cal)
desc_df

,RuleName,Desc
0,B_F_SX_FIVE_RULE_00042,qualificationfactorscore <= 457 and age <= 27 ...
1,B_F_SX_FIVE_RULE_00043,gzfm_0012 == 1 and tdimbankstackingscore >= 0 ...
2,B_F_SX_FIVE_RULE_00044,gzfm_0012 == 1 and antgroupinsightscore > 57.0...
3,B_F_SX_FIVE_RULE_00045,antgroupinsightscore > 76.174 and gender == 0 ...
4,B_F_SX_FIVE_RULE_00046,channelpromote in ('百融') and br_als_m3_id_caon...
...,...,...
182,CREDIT_PEOPLE_BANK_RULE_PB_FLAGB2_00045,ishavecreditreport == 1 and islaunchchannel ==...
183,CREDIT_PEOPLE_BANK_RULE_PB_FLAGB2_00046,ishavecreditreport == 1 and islaunchchannel ==...
184,CREDIT_PEOPLE_BANK_RULE_PB_FLAGB2_00047,ishavecreditreport == 1 and islaunchchannel ==...
185,CREDIT_PEOPLE_BANK_RULE_PB_FLAGB2_00050,ishavecreditreport == 1 and islaunchchannel ==...


In [70]:
rule_list2=pd.merge(left=rule_list,right=desc_df,on='RuleName')
rule_list2

,RuleName,Enabled,Content,Desc
0,B_F_SX_THIRD_RULE_00001,true,百融限高版拒绝,br_flag_executionlimited == 1
1,B_F_SX_THIRD_RULE_00002,true,拒绝-数盒魔方-停机次数,"phoneshutdown in ('(-1,-1]', '(5,+]', '(5,+)')"
2,B_F_SX_THIRD_RULE_00003,true,"（融360,数票通,洋钱罐,百融,拍拍贷联营,OPPO ,优速贷,花鸭,AI营销,同程半流程...",br_als_m1_id_caon_allnum >= 16 and bh_revremai...
3,B_F_SX_THIRD_RULE_00004,true,"（融360,数票通,洋钱罐,百融,拍拍贷联营,OPPO ,优速贷,花鸭,AI营销,同程半流程...",br_als_m1_id_caon_allnum >= 16 and age <= 30 a...
4,B_F_SX_THIRD_RULE_00005,true,"（融360,数票通,洋钱罐,百融,拍拍贷联营,OPPO,优速贷,花鸭,AI营销,同程半流程,...",br_als_d7_id_nbank_else_orgnum > 5 and bh_revr...
...,...,...,...,...
182,B_F_SX_SECOND_RULE_00006,true,百行-循环-历史最严重逾期状态（过去1080 天）为M2/M3/M4/M5/M6/M6/M7,bh_revmaxoverduestatus_sum == 'M2' or bh_revma...
183,B_F_SX_SECOND_RULE_00007,true,百行-非循环-历史最严重逾期状态（过去1080 天）为M2/M3/M4/M5/M6/M7,bh_nonrevmaxoverduestatus_sum == 'M2' or bh_no...
184,B_F_SX_SECOND_RULE_00008,true,百行-金融租赁-历史最严重逾期状态（过去1080 天）为M2/M3/M4/M5/M6/M7,bh_finlsemaxoverduestatus_sum == 'M2' or bh_fi...
185,CREDIT_PEOPLE_BANK_RULE_PB_00252,true,(百融)海纳-风险等级指数 > 3 & 榕树申请评分模型V2411 > 0.0599 & 码...,fxdjzs > 3 and rsmainapplyfinalscorev2411 > 0....


In [71]:
#查看特定规则
rule_cal[rule_cal['RuleName']=='Credit_ZA_N012']

,RuleName,Enabled,ConditionType,VariableCategory,Variable,VariableLabel,Datatype,Value,LogicalOperator,Depth


In [72]:
#人工规则后续手动打标
rule_list2[rule_list2['Desc']=='人工规则']

,RuleName,Enabled,Content,Desc


In [73]:
rule_list2.to_excel(path+'现有规则整理2.xlsx',index=False)

# 生成sql代码
样本需要包含：   
主键：credit_no, cash_no, app_no等  
x：规则中用到的各种变量值  

In [20]:
#非人行变量
notpb_var_list = pd.DataFrame(rule_cal[rule_cal['VariableCategory']!='人行数据采集']['Variable'].unique(),columns=['VarName'])
notpb_var_list['varname']=notpb_var_list['VarName'].apply(lambda x:x.lower())
notpb_var_list[",'VarName'"]=notpb_var_list['VarName'].apply(lambda x:",'"+str(x)+"'")
notpb_var_list["VarName as varname"]=notpb_var_list[",'VarName'"]+' as '+notpb_var_list['varname']
notpb_var_list['var_tb.varname']=',var_tb.'+notpb_var_list['varname']
notpb_var_list.iloc[0, 2:4] = notpb_var_list.iloc[0, 2:4].str[1:]
notpb_var_list

,VarName,varname,",'VarName'",VarName as varname,var_tb.varname
0,br_flag_executionlimited,br_flag_executionlimited,'br_flag_executionlimited','br_flag_executionlimited' as br_flag_executio...,",var_tb.br_flag_executionlimited"
1,phoneShutDown,phoneshutdown,",'phoneShutDown'",",'phoneShutDown' as phoneshutdown",",var_tb.phoneshutdown"
2,br_als_m1_id_caon_allnum,br_als_m1_id_caon_allnum,",'br_als_m1_id_caon_allnum'",",'br_als_m1_id_caon_allnum' as br_als_m1_id_ca...",",var_tb.br_als_m1_id_caon_allnum"
3,bh_RevremainingAmount_sum,bh_revremainingamount_sum,",'bh_RevremainingAmount_sum'",",'bh_RevremainingAmount_sum' as bh_revremainin...",",var_tb.bh_revremainingamount_sum"
4,channelPromote,channelpromote,",'channelPromote'",",'channelPromote' as channelpromote",",var_tb.channelpromote"
...,...,...,...,...,...
75,bh_RevmaxOverdueStatus_sum,bh_revmaxoverduestatus_sum,",'bh_RevmaxOverdueStatus_sum'",",'bh_RevmaxOverdueStatus_sum' as bh_revmaxover...",",var_tb.bh_revmaxoverduestatus_sum"
76,bh_nonRevmaxOverdueStatus_sum,bh_nonrevmaxoverduestatus_sum,",'bh_nonRevmaxOverdueStatus_sum'",",'bh_nonRevmaxOverdueStatus_sum' as bh_nonrevm...",",var_tb.bh_nonrevmaxoverduestatus_sum"
77,bh_finLseMaxOverdueStatus_sum,bh_finlsemaxoverduestatus_sum,",'bh_finLseMaxOverdueStatus_sum'",",'bh_finLseMaxOverdueStatus_sum' as bh_finlsem...",",var_tb.bh_finlsemaxoverduestatus_sum"
78,fxdjzs,fxdjzs,",'fxdjzs'",",'fxdjzs' as fxdjzs",",var_tb.fxdjzs"


In [21]:
#人行变量
pb_var_list = pd.DataFrame(rule_cal[rule_cal['VariableCategory']=='人行数据采集']['Variable'].unique(),columns=['VarName'])
pb_var_list['varname']=pb_var_list['VarName'].apply(lambda x:x.lower())
pb_var_list[",'VarName'"]=pb_var_list['VarName'].apply(lambda x:",'"+str(x)+"'")
pb_var_list["VarName as varname"]=pb_var_list[",'VarName'"]+' as '+pb_var_list['varname']
pb_var_list['pb_var_tb.varname']=',pb_var_tb.'+pb_var_list['varname']
pb_var_list.iloc[0, 2:4] = pb_var_list.iloc[0, 2:4].str[1:]
pb_var_list

,VarName,varname,",'VarName'",VarName as varname,pb_var_tb.varname
0,isHaveCreditReport,ishavecreditreport,'isHaveCreditReport','isHaveCreditReport' as ishavecreditreport,",pb_var_tb.ishavecreditreport"
1,PB_CreTra_Bas1LaF_n3_NFYDebitCard1to7BWorstState,pb_cretra_bas1laf_n3_nfydebitcard1to7bworststate,",'PB_CreTra_Bas1LaF_n3_NFYDebitCard1to7BWorstS...",",'PB_CreTra_Bas1LaF_n3_NFYDebitCard1to7BWorstS...",",pb_var_tb.pb_cretra_bas1laf_n3_nfydebitcard1t..."
2,PB_Pco_OveS_DCNFiveYLongOverdueM,pb_pco_oves_dcnfiveylongoverduem,",'PB_Pco_OveS_DCNFiveYLongOverdueM'",",'PB_Pco_OveS_DCNFiveYLongOverdueM' as pb_pco_...",",pb_var_tb.pb_pco_oves_dcnfiveylongoverduem"
3,PB_Pco_OveS_RCSNFiveYOverdueAccNum,pb_pco_oves_rcsnfiveyoverdueaccnum,",'PB_Pco_OveS_RCSNFiveYOverdueAccNum'",",'PB_Pco_OveS_RCSNFiveYOverdueAccNum' as pb_pc...",",pb_var_tb.pb_pco_oves_rcsnfiveyoverdueaccnum"
4,PB_Pco_OveS_NRLNFiveYOverdueAccNum,pb_pco_oves_nrlnfiveyoverdueaccnum,",'PB_Pco_OveS_NRLNFiveYOverdueAccNum'",",'PB_Pco_OveS_NRLNFiveYOverdueAccNum' as pb_pc...",",pb_var_tb.pb_pco_oves_nrlnfiveyoverdueaccnum"
...,...,...,...,...,...
75,PB_Pqo_QueR_n3_TotalQueSetupTypeNum_DAll,pb_pqo_quer_n3_totalquesetuptypenum_dall,",'PB_Pqo_QueR_n3_TotalQueSetupTypeNum_DAll'",",'PB_Pqo_QueR_n3_TotalQueSetupTypeNum_DAll' as...",",pb_var_tb.pb_pqo_quer_n3_totalquesetuptypenum..."
76,mg_dk_x1845,mg_dk_x1845,",'mg_dk_x1845'",",'mg_dk_x1845' as mg_dk_x1845",",pb_var_tb.mg_dk_x1845"
77,PB_Pqo_QueR_n1_TotalQueSetupTypeNum_DAll,pb_pqo_quer_n1_totalquesetuptypenum_dall,",'PB_Pqo_QueR_n1_TotalQueSetupTypeNum_DAll'",",'PB_Pqo_QueR_n1_TotalQueSetupTypeNum_DAll' as...",",pb_var_tb.pb_pqo_quer_n1_totalquesetuptypenum..."
78,mg_dk_x1247,mg_dk_x1247,",'mg_dk_x1247'",",'mg_dk_x1247' as mg_dk_x1247",",pb_var_tb.mg_dk_x1247"


In [24]:
#主表，可以增加任意你想要的字段
main_query='''
select credit_no,init_date,is_credit_pass,customer_group,id_no,mobile
from dwd.xxy_credit_mid_table_inc
where init_date between '2023-09-01' and '2024-01-31'
and channel_promote = '洋钱罐'
'''

In [28]:
main_query='''
select credit_no,init_time,init_date,is_init_credit,is_credit_pass,is_entry_outer_exam,customer_group,channel_promote,id_no,mobile
from dwd.xxy_credit_mid_table_inc
where channel_promote = '众安小贷'
and init_date>='2024-11-24'
and credit_no in (
    select credit_no from dwd.xxyrmps_tm_credit_history_inc
    where dt>='2024-11-24'
    and node_name='第五轮授信规则检测'
)
and credit_no in (
    select credit_no from dwd.xxyrmps_tm_credit_main_ext_attr_inc
    where dt>='2024-11-24'
    and attr_name = 'mktFlag'
    and string_value = 1
)
'''

In [22]:
main_query = '''
	        SELECT credit_no 
			from dwd.risk_overdue_fpd_daily_full
			where dt=date_sub(current_date(),1)
			and cash_channel_promote ='百融'
			and app_type ='NEW'
			and cash_complete_month >='2024-01'
			and cash_complete_date < '2024-10-19'
'''

In [23]:
# 提取需要的值
var_tb_varname = '\n'.join(notpb_var_list["var_tb.varname"].astype(str))
pb_var_tb_varname = '\n'.join(pb_var_list["pb_var_tb.varname"].astype(str))
notpb_varname = '\n'.join(notpb_var_list[",'VarName'"].astype(str))
pb_varname = '\n'.join(pb_var_list[",'VarName'"].astype(str))
notpb_varname_as = '\n'.join(notpb_var_list["VarName as varname"].astype(str))
pb_varname_as = '\n'.join(pb_var_list["VarName as varname"].astype(str))

In [24]:
# SQL 查询模板
sql_query = f'''
SELECT main.*
{var_tb_varname}
{pb_var_tb_varname}
FROM (
{main_query}
) AS main
LEFT JOIN (
    SELECT * 
    FROM (
        SELECT m.credit_no, attr_name, IFNULL(int_value, IFNULL(decimal_value, string_value)) AS key_value
        FROM (
{main_query}
        ) AS m
        LEFT JOIN dwd.xxyrmps_tm_credit_main_ext_attr_inc AS t ON t.credit_no = m.credit_no
        WHERE attr_name IN (
{notpb_varname}
        )
    )
    PIVOT( MAX(key_value) FOR (attr_name) IN (
{notpb_varname_as}
    )) 
) AS var_tb ON var_tb.credit_no = main.credit_no
LEFT JOIN (
    SELECT * 
    FROM (
        SELECT m.credit_no, attr_name, IFNULL(decimal_value, IFNULL(string_value, int_value)) AS key_value
        FROM (
{main_query}
        ) AS m
        LEFT JOIN ods.xxyrmps_tm_people_bank_ext_attr_inc AS t ON t.app_no = m.credit_no
        WHERE attr_name IN (
{pb_varname}
        )
    )
    PIVOT( MAX(key_value) FOR (attr_name) IN (
{pb_varname_as}
    )) 
) AS pb_var_tb ON pb_var_tb.credit_no = main.credit_no
'''

print(sql_query)


SELECT main.*
,var_tb.br_flag_executionlimited
,var_tb.phoneshutdown
,var_tb.br_als_m1_id_caon_allnum
,var_tb.bh_revremainingamount_sum
,var_tb.channelpromote
,var_tb.age
,var_tb.br_als_d7_id_nbank_else_orgnum
,var_tb.bh_revcreditlimitsum_d180
,var_tb.br_sl_id_court_bad
,var_tb.br_sl_id_court_executed
,var_tb.br_sl_id_bank_lost
,var_tb.br_sl_id_nbank_lost
,var_tb.br_sl_cell_bank_lost
,var_tb.br_sl_cell_nbank_lost
,var_tb.br_als_m3_cell_nbank_orgnum
,var_tb.br_als_m12_id_pdl_orgnum
,var_tb.br_als_m12_id_nbank_cons_allnum
,var_tb.br_als_m12_cell_rel_allnum
,var_tb.br_als_m12_cell_pdl_allnum
,var_tb.bh_queryhistory_cnt
,var_tb.br_alm_m3_cell_nbank_oth_orgnum
,var_tb.bh_nonrevmaxloanamount_d360
,var_tb.br_als_lst_id_bank_inteday
,var_tb.br_als_m1_id_nbank_nsloan_orgnum
,var_tb.brmultiscoreyqglevel
,var_tb.brmultiscore
,var_tb.bhcreditscorev1
,var_tb.br_als_d7_id_nbank_orgnum
,var_tb.qualificationfactorscore
,var_tb.gzfm_0012
,var_tb.tdimbankstackingscore
,var_tb.antgroupinsightscore
,var_

In [44]:
# 保存为 .sql 文件
# sql_name='./output/Spark取数代码.sql'
# with open(sql_name, 'w', encoding='utf-8') as file:
#     file.write(sql_query)

# print(f"SQL query saved to {sql_name}")

SQL query saved to ./output/Spark取数代码.sql


# 现有策略遍历

In [74]:
def rule_impact(df, rule_desc, target_column='y', positive_label=1):
    """
    分析规则在数据集中的命中情况，并计算bad_rate和lift等指标。

    参数:
    df : pd.DataFrame
        包含待分析数据的DataFrame，至少需要包含rule_desc所涉及的字段和target_column。
    rule_desc : str
        规则的逻辑表达式，使用df.query(rule_desc)进行命中样本的筛选。
    target_column : str, optional
        目标列，表示贷后结果（如违约标志），默认为'y'。
    positive_label : int or str, optional
        目标列中表示“bad”样本的值（如违约的标签），默认为1。

    返回:
    result : pd.DataFrame
        返回一个包含命中、不命中和总体的分析结果，包括数量、占比、bad_rate和lift。
    """
    # 总体样本数量
    total_count = df.shape[0]
    
    # 命中规则的样本
    hit_df = df.query(rule_desc)
    hit_count = hit_df.shape[0]
    
    # 不命中规则的样本
    miss_df = df[~df.index.isin(hit_df.index)]
    miss_count = miss_df.shape[0]
    
    # 计算各类样本的违约率（bad_rate）
    total_bad_rate = df[target_column].value_counts(normalize=True).get(positive_label, 0)
    hit_bad_rate = hit_df[target_column].value_counts(normalize=True).get(positive_label, 0)
    miss_bad_rate = miss_df[target_column].value_counts(normalize=True).get(positive_label, 0)
    
    # 计算各类样本的bad_cnt
    total_bad_cnt = total_count*total_bad_rate
    hit_bad_cnt = hit_count*hit_bad_rate
    miss_bad_cnt = miss_count*miss_bad_rate
    
    # Lift = 命中bad_rate / 总体bad_rate
    lift = hit_bad_rate / total_bad_rate if total_bad_rate != 0 else float('inf')
    
    # 构建结果表
    result = pd.DataFrame({
        'Group': ['Hit', 'Miss', 'Total'],
        'Count': [hit_count, miss_count, total_count],
        'Percentage': [hit_count / total_count, miss_count / total_count, 1],
        'Bad':[hit_bad_cnt, miss_bad_cnt, total_bad_cnt],
        'Bad_Rate': [hit_bad_rate, miss_bad_rate, total_bad_rate],
        'Lift': [lift, '-', '-']
    })

    return result


In [118]:
df = pd.read_excel(path+'榕树打标底表--20250208.xlsx')
df

,credit_no,cash_no,init_credit_date,init_credit_month,cash_complete_date,cash_complete_month,cash_channel_promote,loan_amount,def_fpd30,agr_fpd30,...,mg_base_x46_7,mg_dk_x2110,mg_jh_x120,mg_dk_x1370,mg_dkd1_x844,mg_dk_x2405,PB_Pqo_QueR_n3_TotalQueSetupNum_DAll,PB_Pqo_QueR_n3_TotalQueSetupTypeNum_DAll,mg_dk_x1845,mg_dkd1_x31
0,2023101919572972163,2023101919572972163,2023-10-19,2023-10,2023-11-20,2023-11,百融,10000,0,1,...,0.0,0.0000,0.0,222099.0,0.000,50000.0,0.214300,0.428600,27111.0,0.0283
1,2023101920176070355,2023121115196849173,2023-10-19,2023-10,2023-12-11,2023-12,百融,11600,0,1,...,0.0,0.9840,0.0,180616.0,0.811,60000.0,0.414600,0.800000,14616.0,0.5967
2,2023101920322437413,2023120321423245870,2023-10-19,2023-10,2023-12-04,2023-12,百融,3600,0,1,...,1.0,0.9987,2.0,16712.0,0.970,35600.0,0.216200,0.500000,776000.0,0.0083
3,2023102014050580791,2023102014050580791,2023-10-20,2023-10,2023-10-20,2023-10,百融,10000,0,1,...,0.0,0.0000,1.0,323274.0,0.000,200000.0,0.240700,0.571400,13964.0,0.1411
4,2023102014061771435,2023102014061771435,2023-10-20,2023-10,2023-10-20,2023-10,百融,11000,0,1,...,0.0,0.9267,0.0,49680.0,0.000,24000.0,0.432400,0.666700,870.0,0.0689
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60257,2024101922289444120,2024102019247933745,2024-10-19,2024-10,2024-10-20,2024-10,百融,3000,0,1,...,0.0,0.0000,2.0,123597.0,0.000,40000.0,0.341463,0.571429,818.0,1.0000
60258,2024101922289444120,2024102617055051522,2024-10-19,2024-10,2024-10-26,2024-10,百融,3000,0,1,...,0.0,0.0000,2.0,123597.0,0.000,40000.0,0.341463,0.571429,818.0,1.0000
60259,2024101922405638055,2024101922423662643,2024-10-19,2024-10,2024-10-19,2024-10,百融,5300,0,1,...,0.0,0.9328,1.0,311831.0,0.000,60750.0,0.305085,0.571429,5604.0,0.0659
60260,2024101923190818635,2024102301335440284,2024-10-19,2024-10,2024-10-23,2024-10,百融,5000,0,1,...,0.0,0.9747,0.0,89861.0,0.000,71500.0,0.387097,0.714286,6651.0,0.0106


In [121]:
# 将列名转换为小写
df.columns = df.columns.str.lower()

In [122]:
df['channelpromote']=df['cash_channel_promote']

In [123]:
df.shape[0]

60262

In [124]:
df_sample = df[(df['init_credit_date']<='2024-10-19')&(df['agr_mob4_30+']>=1)].copy()
df_sample.shape[0]

53736

In [125]:
df_sample['channelpromote']

0        百融
1        百融
2        百融
3        百融
4        百融
         ..
56392    百融
56397    百融
56398    百融
56400    百融
56401    百融
Name: channelpromote, Length: 53736, dtype: object

In [126]:
collected_results = []

# 遍历规则，筛选出Enabled为'true'，且RuleName不以'T'结尾的规则  
# for rule_name, desc in rule_list2[(rule_list2['Enabled'] == 'true') & (rule_list2['RuleName'].str[-1:] != 'T')][['RuleName', 'Desc']].values:
for rule_name, desc in rule_list2[['RuleName', 'Desc']].values:
    try:
        # 调用 rule_impact 函数，并获取结果
        result = rule_impact(df_sample, desc, 'def_mob4_30+', 1) #这里要改y值

        # 获取结果的第一行
        rule_result = result.iloc[0]

        # 在 rule_result 中加入 rule_name 和 desc
        rule_result = rule_result.copy()
        rule_result['RuleName'] = rule_name
        rule_result['Desc'] = desc

        # 增加 bad_cnt 列，计算 Count * Bad_Rate
        rule_result['Bad_Cnt'] = rule_result['Count'] * rule_result['Bad_Rate']

        # 将修改后的结果添加到收集器中
        collected_results.append(rule_result)

    except Exception as e:
        print(f"规则 '{rule_name}' 执行时出错: {e}")
        continue  # 跳过出错的规则

# 将收集到的结果合并成一个新的 DataFrame
if collected_results:
    final_df = pd.DataFrame(collected_results, columns=['RuleName', 'Desc', 'Count', 'Percentage','Bad_Cnt', 'Bad_Rate', 'Lift'])
else:
    final_df = pd.DataFrame()  # 如果没有结果，则返回空DataFrame


In [127]:
final_df

,RuleName,Desc,Count,Percentage,Bad_Cnt,Bad_Rate,Lift
0,B_F_SX_THIRD_RULE_00001,br_flag_executionlimited == 1,0,0.000000,0.0,0.000000,0.000000
0,B_F_SX_THIRD_RULE_00002,"phoneshutdown in ('(-1,-1]', '(5,+]', '(5,+)')",45,0.000837,1.0,0.022222,0.530019
0,B_F_SX_THIRD_RULE_00003,br_als_m1_id_caon_allnum >= 16 and bh_revremai...,46,0.000856,2.0,0.043478,1.036994
0,B_F_SX_THIRD_RULE_00004,br_als_m1_id_caon_allnum >= 16 and age <= 30 a...,27,0.000502,1.0,0.037037,0.883365
0,B_F_SX_THIRD_RULE_00005,br_als_d7_id_nbank_else_orgnum > 5 and bh_revr...,1,0.000019,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...
0,B_F_SX_SECOND_RULE_00006,bh_revmaxoverduestatus_sum == 'M2' or bh_revma...,0,0.000000,0.0,0.000000,0.000000
0,B_F_SX_SECOND_RULE_00007,bh_nonrevmaxoverduestatus_sum == 'M2' or bh_no...,0,0.000000,0.0,0.000000,0.000000
0,B_F_SX_SECOND_RULE_00008,bh_finlsemaxoverduestatus_sum == 'M2' or bh_fi...,0,0.000000,0.0,0.000000,0.000000
0,CREDIT_PEOPLE_BANK_RULE_PB_00252,fxdjzs > 3 and rsmainapplyfinalscorev2411 > 0....,0,0.000000,0.0,0.000000,0.000000


In [128]:
result_df=pd.merge(left=rule_list2,right=final_df,on=['RuleName','Desc'],how='left')
result_df

,RuleName,Enabled,Content,Desc,Count,Percentage,Bad_Cnt,Bad_Rate,Lift
0,B_F_SX_THIRD_RULE_00001,true,百融限高版拒绝,br_flag_executionlimited == 1,0,0.000000,0.0,0.000000,0.000000
1,B_F_SX_THIRD_RULE_00002,true,拒绝-数盒魔方-停机次数,"phoneshutdown in ('(-1,-1]', '(5,+]', '(5,+)')",45,0.000837,1.0,0.022222,0.530019
2,B_F_SX_THIRD_RULE_00003,true,"（融360,数票通,洋钱罐,百融,拍拍贷联营,OPPO ,优速贷,花鸭,AI营销,同程半流程...",br_als_m1_id_caon_allnum >= 16 and bh_revremai...,46,0.000856,2.0,0.043478,1.036994
3,B_F_SX_THIRD_RULE_00004,true,"（融360,数票通,洋钱罐,百融,拍拍贷联营,OPPO ,优速贷,花鸭,AI营销,同程半流程...",br_als_m1_id_caon_allnum >= 16 and age <= 30 a...,27,0.000502,1.0,0.037037,0.883365
4,B_F_SX_THIRD_RULE_00005,true,"（融360,数票通,洋钱罐,百融,拍拍贷联营,OPPO,优速贷,花鸭,AI营销,同程半流程,...",br_als_d7_id_nbank_else_orgnum > 5 and bh_revr...,1,0.000019,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
182,B_F_SX_SECOND_RULE_00006,true,百行-循环-历史最严重逾期状态（过去1080 天）为M2/M3/M4/M5/M6/M6/M7,bh_revmaxoverduestatus_sum == 'M2' or bh_revma...,0,0.000000,0.0,0.000000,0.000000
183,B_F_SX_SECOND_RULE_00007,true,百行-非循环-历史最严重逾期状态（过去1080 天）为M2/M3/M4/M5/M6/M7,bh_nonrevmaxoverduestatus_sum == 'M2' or bh_no...,0,0.000000,0.0,0.000000,0.000000
184,B_F_SX_SECOND_RULE_00008,true,百行-金融租赁-历史最严重逾期状态（过去1080 天）为M2/M3/M4/M5/M6/M7,bh_finlsemaxoverduestatus_sum == 'M2' or bh_fi...,0,0.000000,0.0,0.000000,0.000000
185,CREDIT_PEOPLE_BANK_RULE_PB_00252,true,(百融)海纳-风险等级指数 > 3 & 榕树申请评分模型V2411 > 0.0599 & 码...,fxdjzs > 3 and rsmainapplyfinalscorev2411 > 0....,0,0.000000,0.0,0.000000,0.000000


In [129]:
result_df.to_excel(path+'现有规则效果mob4_30+.xlsx',index=False)